In [8]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Dictionary that stores all the emotions with their respective genres

In [24]:
emotion_genre_map_1 = {
    "energetic": ["afrobeat", "breakbeat", "drum-and-bass", "dubstep", "hardcore", "hardstyle", "punk", "ska", "metal-misc", "hip-hop", "reggaeton", "house", "samba", "sertanejo"],
    "calm": ["acoustic", "ambient", "bossanova", "new-age", "piano", "sleep", "rainy-day"],
    "nostalgic": ["anime", "disco", "j-pop", "j-rock", "rock-n-roll", "rockabilly", "grunge", "emo", "indie-pop", "j-idol"],
    "soulful": ["blues", "soul", "funk"],
    "happy": ["party", "pop", "pop-film"],
    "intense": ["black-metal", "death-metal", "grindcore", "heavy-metal", "metalcore"],
    "romantic": ["romance", "love-songs"],
    "introspective": ["alternative", "folk", "singer-songwriter", "songwriter", "british"],
    "upbeat": ["bluegrass", "brazil", "dancehall", "funk", "reggae"],
    "reflective": ["british", "indie-pop", "post-dubstep", "sad", "piano"],
    "brooding": ["alt-rock", "goth", "industrial", "psych-rock"],
    "groovy": ["groove", "club", "dance", "electro"],
    "adventurous": ["dub", "electronic", "idm", "psycho-rock", "rockabilly", "trance"],
    "traditional": ["country", "folklore"],
    "cultural": ["french", "german", "iranian", "malay", "philippines-opm"],
    "energizing": ["techno", "trance", "dubstep", "electronic"],
    "relaxing": ["chill", "easy-listening", "lounge", "soft-rock", "ambient"],
    "uplifting": ["happy", "power-pop", "k-pop"],
    "emotional": ["r-n-b", "soul"],
    "mysterious": ["instrumental", "jazz"],
    "fiery": ["flamenco", "spanish"],
    "reflective": ["acoustic", "chill", "classical", "indie-folk", "jazz", "rainy-day", "sleep"],
    "motivational": ["motivation", "work-out"],
    "sad": ["emo", "indie", "sad", "indie-pop", "singer-songwriter", "acoustic", "piano", "rainy-day"]
}

## Functions for embedding input and matching the embeddings with the proper emotion and genre

In [3]:
def embedd_input(input):
    model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )
    model.eval()
    tkn = BertTokenizer.from_pretrained('bert-base-uncased')

    episode_summaries = [input]
    eps_tokenized = []

    for ep in episode_summaries:
        marked_text = "[CLS] " + str(ep) + " [SEP]"
        tokenized = tkn.tokenize(marked_text)
        eps_tokenized.append(tokenized)

    eps_tokenized

    eps_embeddings = []


    start = 1
    max_length = 512  
    count = 0

    for token_s in eps_tokenized:
    # Turns our tokens into their index in the corpus
        indexed_tokens = tkn.convert_tokens_to_ids(token_s)
    
    # Truncate the sequence if it's longer than the max length
        indexed_tokens = indexed_tokens[:max_length]
        segments_ids = [start] * len(indexed_tokens)
    
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

    # Generates the embedding
        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]

        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
    
    # Adding the embedding to our list, more specifically, the embedding of the first and last token
        token_vecs = hidden_states[-2][0]
        eps_embeddings.append(token_vecs)
        count +=1
        if count % 1000 == 0:
            print(count)

    

        
    embedded_eps = []
    for vec in eps_embeddings:
        sentence_embedding = torch.mean(vec, dim= 0)
        embedded_eps.append(sentence_embedding)


#token_vecs = hidden_states[-2][0]
#sentence_embedding = torch.mean(token_vecs, dim=0)

    return embedded_eps

    

    
    


In [9]:
def emotion_comparision(inpt):

  
  emotions = ['energetic','calm','nostalgic', 'soulful', 'happy', 'intense', 'romantic', 'introspective', 'upbeat', 'reflective', 'brooding', 'playful', 'groovy', 'emotional', 'adventurous', 'traditional',
 'cultural','energizing','relaxing','uplifting', 'sensual', 'mysterious', 'fiery', 'exotic', 'motivational', 'sad']
  loaded_embeddings_np = np.load('emotion_embeddings.npy', allow_pickle=True)
  sentence_np = embedd_input(inpt)[0].numpy()
  loaded_embeddings_np = np.load('emotion_embeddings.npy', allow_pickle=True)
  similarity_scores = cosine_similarity([sentence_np], loaded_embeddings_np)[0]
  top_3_indices = np.argsort(similarity_scores)[-3:][::-1]
  top_3_emotions = [(emotions[index], similarity_scores[index]) for index in top_3_indices]
  rtn = []
  for x in top_3_emotions:
    rtn.append(emotion_genre_map_1[x[0]])


  return rtn

In [26]:
emotion_comparision("I am so excited, I won so much money!")

[['party', 'pop', 'pop-film'],
 ['blues', 'soul', 'funk'],
 ['motivation', 'work-out']]